In [18]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt

UBIT = 'mgosi'
np.random.seed(sum([ord(c) for c in UBIT]))

# Task 1

# Creating SIFT Images

In [19]:
img1 = cv2.imread('mountain1.jpg')
img2 = cv2.imread('mountain2.jpg')

#Converting to Grayscale
gray_1= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray_2= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

#Initializing Sift
sift = cv2.xfeatures2d.SIFT_create()

#Applying Sift Algorithm to find the keypoints and their descriptors
kp_1, desc_1 = sift.detectAndCompute(gray_1,None)
kp_2, desc_2 = sift.detectAndCompute(gray_2,None)

#Drawing corresponding Keypoints
img_1=cv2.drawKeypoints(gray_1,kp_1,img_1)
img_2=cv2.drawKeypoints(gray_2,kp_2,img_2)

cv2.imwrite('task1_sift1.jpg',img_1)
cv2.imwrite('task1_sift2.jpg',img_2)

True

# Matching and Homography

In [20]:
#Using Brute Force matcher

bf = cv2.BFMatcher()

#It tries to find the closest descriptor in the second set , and applyes KNN Matching to the descriptors
matches = bf.knnMatch(desc_1, desc_2, k= 2)

# store all the good_matches matches as per Lowe's ratio test.
good_matches = []
for m,n in matches:
    if m.distance < 0.75*n.distance:      #Good matching criteria
        good_matches.append(m)


src_pts = np.float32([ kp_1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)             #Gives us the index of the descriptor in the list of train descriptors 
dst_pts = np.float32([ kp_2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)               #Gives us the index of the descriptor in the list of query descriptors 

#homography relates the transformation between two plane by using RANSAC Algorithm
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()
print (H)

[[ 1.58720376e+00 -2.91747553e-01 -3.95226519e+02]
 [ 4.48097764e-01  1.43063310e+00 -1.90273584e+02]
 [ 1.20808480e-03 -6.07787702e-05  1.00000000e+00]]


# Knn Matching

In [21]:
ran_pts1 = []
ran_pts2 = []
for i in [np.random.randint(0,len(good_matches) -1 ) for x in range(10)] :
    ran_pts1.append(good_matches[i])
    ran_pts2.append(matchesMask[i])

#Plotting all the matches
img_3 = cv2.drawMatches(img_1,kp_1,img_2,kp_2,good_matches,None,flags=2)
cv2.imwrite("task1_matches_knn.jpg",img_3)

True

# Matching 10 Points

In [16]:
#parameters to draw the 10 random points and only inliners
params = dict(matchColor = np.random.randint(low = 0, high = 255, size = 3).tolist(),
                   matchesMask = ran_pts2, # draw only inliners
                   flags = 2)

#Plotting keypoints for only 10 Inliers
img_4 = cv2.drawMatches(img_1,kp_1,img_2,kp_2,ran_pts1,None,**params)
cv2.imwrite("task1_matches.jpg",img_4)

True

# Warping

In [17]:
height,width = img_1.shape[:2]
pts = np.float32([ [0,0],[0,height-1],[width-1,height-1],[width-1,0] ]).reshape(-1,1,2)
#Taking the border points of the image(i.e Corner Points)
dst = cv2.perspectiveTransform(pts,H)

#Extracting the rows and cols
rows1, cols1 = img_1.shape[:2]
rows2, cols2 = img_2.shape[:2]

#Getting Border Points
pts1 = np.float32([[0,0], [0,rows1], [cols1, rows1], [cols1,0]]).reshape(-1,1,2)
temp_points = np.float32([[0,0], [0,rows2], [cols2, rows2], [cols2,0]]).reshape(-1,1,2)

#Taking Perspective
pts2 = cv2.perspectiveTransform(temp_points, H)
pts = np.concatenate((pts1, pts2), axis=0)

#For calculating the size of the output image
[x_min, y_min] = np.int32(pts.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(pts.max(axis=0).ravel() + 0.5)

trans_dist = [-x_min, -y_min]

H_trans = np.array([[1, 0, trans_dist[0]], [0, 1, trans_dist[1]], [0,0,1]])
#Warping image 1 on top of Image 2
warp_img = cv2.warpPerspective(img1, H_trans.dot(H), (x_max - x_min, y_max - y_min))
warp_img[trans_dist[1]:rows1+trans_dist[1],trans_dist[0]:cols1+trans_dist[0]] = img2

cv2.imwrite("task1_pano.jpg",warp_img)

True